# Import Libraries


In [20]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import requests
import os
import gdown
import sys

In [22]:
url = sys.argv[1]
file_id = url.split('/')[-1]
prefix = 'https://drive.google.com/drive/u/0/folders/17WG20uUt9IwawMqFtmeBD9EieoMcWnG1'
gdown.download(prefix+file_id)

c:\Users\tedjo\anaconda3\Lib\site-packages\gdown\parse_url.py:48: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=None
  warnings.warn(
Downloading...
From: https://drive.google.com/drive/u/0/folders/17WG20uUt9IwawMqFtmeBD9EieoMcWnG1--f=c:\Users\tedjo\AppData\Roaming\jupyter\runtime\kernel-v2-9772DDaNpKRL0oER.json
To: c:\Users\tedjo\OneDrive\Documents\Beyond Words\Projects\Dance Daily\dancevibedaily\kernel-v2-9772DDaNpKRL0oER.json
1.21MB [00:00, 6.11MB/s]


'kernel-v2-9772DDaNpKRL0oER.json'

# Media files

In [15]:
import gdown

ModuleNotFoundError: No module named 'gdown'

# DataFrames

# Data Cleaning